In [3]:
from __future__ import print_function, division
#virtualenv -p python3 venv

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [5]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

In [6]:
ls ../data/train/

Type_1/  Type_2/  Type_3/


In [7]:
df = c.data.make_base_df()
df = c.processing.transform_save_imgs(df, c.processing.resize_n, n=224);

In [8]:
train, validate = c.data.split_df(df); 

In [9]:
print(train.shape)
train.head()

(1332, 5)


,path,Type,filetype,num_id,processed_path
137,../data/train/Type_1/102.jpg,1,jpg,102,../data/processed/resize_224/train/102.jpg
130,../data/train/Type_1/1105.jpg,1,jpg,1105,../data/processed/resize_224/train/1105.jpg
1032,../data/train/Type_3/573.jpg,3,jpg,573,../data/processed/resize_224/train/573.jpg
177,../data/train/Type_1/1389.jpg,1,jpg,1389,../data/processed/resize_224/train/1389.jpg
1303,../data/train/Type_3/1048.jpg,3,jpg,1048,../data/processed/resize_224/train/1048.jpg


In [10]:
print(validate.shape)
validate.head()

(149, 5)


,path,Type,filetype,num_id,processed_path
1453,../data/train/Type_3/670.jpg,3,jpg,670,../data/processed/resize_224/train/670.jpg
1202,../data/train/Type_3/427.jpg,3,jpg,427,../data/processed/resize_224/train/427.jpg
1403,../data/train/Type_3/635.jpg,3,jpg,635,../data/processed/resize_224/train/635.jpg
162,../data/train/Type_1/230.jpg,1,jpg,230,../data/processed/resize_224/train/230.jpg
625,../data/train/Type_2/1329.jpg,2,jpg,1329,../data/processed/resize_224/train/1329.jpg


# Model creation

In [22]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False)

# remove most of VGG16 to leave only the first two conv blocks (basic features)
#ie remove top 12 layers
#for _ in range(12):
#    base_model.layers.pop()
#base_model = Model(base_model.input, base_model.layers[-1].output)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
    # we have three classes so usung 3 on dense predictions
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [24]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_3
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d_3
20 dense_5
21 dense_6


In [25]:
#Choosing to train blocks 4 and 5
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

In [26]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 input_3 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True
19 global_average_pooling2d_3 True
20 dense_5 True
21 dense_6 True


In [27]:
# compile the model (should be done *after* setting layers to non-trainable)
# TODO: loss function should be log loss. 'categorical_crossentropy' / c.submission.keras_log_loss
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')
#model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [28]:
features, labels = c.processing.df_to_training_tuples(train, (224, 224, 3), grayscale=False)
val_data = c.processing.df_to_training_tuples(validate, (224,224, 3), grayscale=False)

In [29]:
history = c.analysis.History()

In [30]:
# train the model on the new data for a few epochs
import keras
print('backend : '+keras.backend.backend())
model_history = model.fit(features, labels, batch_size=34, epochs=100, validation_data=val_data,
                          callbacks=[history, keras.callbacks.EarlyStopping(patience=5)])


backend : tensorflow
Train on 1332 samples, validate on 149 samples
Epoch 1/100
1332/1332 [==============================] - 32s - loss: 1.4382 - val_loss: 1.0432
Epoch 2/100
 374/1332 [=======>......................] - ETA: 21s - loss: 0.9463

KeyboardInterrupt: 

In [ ]:
c.analysis.plot_log_errors(history)

In [ ]:
batch_size_t = 1
predictions = model.predict_generator(c.processing.df_to_keras_generator(test, batch_size_t, grayscale=False)
                    ,len(test))
predictions

In [ ]:
test_p = c.processing.append_probabilities(test, predictions, ['1','2','3']); test_p.head()

In [ ]:
test_p.Type.value_counts()

In [ ]:
predictions.sum(0)

In [ ]:
score, ldf = c.submission.compute_losses(test_p)

In [ ]:
score

In [ ]:
ldf.sort_values('Type_1', ascending=False)

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[0])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).processed_path.iloc[0])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[1])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).processed_path.iloc[1])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).path.iloc[3])

In [ ]:
c.data.check_image(ldf.sort_values('log_l', ascending=False).processed_path.iloc[3])